# 转换并量化中文Alpaca Plus模型

这是和知乎系列精调文章

* [尝试对Chinese-LLaMA-Alpaca进行微调-准备篇](https://zhuanlan.zhihu.com/p/630522733)
* [尝试对Chinese-LLaMA-Alpaca进行微调-数据准备篇](https://zhuanlan.zhihu.com/p/630544641)
* [尝试对Chinese-LLaMA-Alpaca进行微调-实践篇](https://zhuanlan.zhihu.com/p/632291297)


对应的在线脚本。

和官方脚本相比增加了对于模型的存储过程，以便我们可以在后续的步骤中进行进一步的精调。

官方脚本中有很多关于Google Colab花费，经验等的介绍，大家可以多多查阅

[官方脚本](https://github.com/ymcui/Chinese-LLaMA-Alpaca/blob/main/notebooks/convert_and_quantize_chinese_alpaca_plus.ipynb)

## 安装相关依赖

和官方不同的是这个地方会多安装一个mpi4py，用于精调阶段

In [ ]:
!pip install torch
!pip install transformers
!pip install git+https://github.com/huggingface/peft
!pip install sentencepiece
!pip install mpi4py

## 克隆目录和代码

In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca

# 完成Google Drive的加载

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 合并模型（Alpaca-Plus-7B）

## 注意：`本步骤只执行一次`

本步骤是对模型进行合并，请注意，这个步骤在我们整体精调步骤中只会执行一次，从第二次开始，都直接跳过这个步骤，直接从Google Drive中加载模型

## 提醒：除精调阶段外，其他步骤可选用花费较少的机型

如果你已经了解到整个流程其实分成了第一次合并，获得中文合并模型，和之后的精调阶段，那么你第一次执行的时候就可以不用选择太贵的硬件，合并不需要太多GPU资源

## 官方提示

💡 转换13B模型提示：
- 请将参数`--base_model`和`--lora_model`中的的`7b`改为`13b`即可
- **免费用户必须增加一个参数`--offload_dir`以缓解内存压力**，例如`--offload_dir ./offload_temp`

该过程比较耗时（下载+转换），需要几分钟到十几分钟不等，请耐心等待。
转换好的模型存放在`alpaca-combined`目录。
如果你不需要量化模型，那么到这一步就结束了。

In [ ]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora.py \
    --base_model decapoda-research/llama-7b-hf \
    --lora_model ziqingyang/chinese-llama-plus-lora-7b,ziqingyang/chinese-alpaca-plus-lora-7b \
    --output_type huggingface \
    --output_dir alpaca-combined

## 保存合并好的模型

这个步骤将合并好的模型拷贝保存到Google Drive中，以便在以后的精调工作中使用，请注意，模型比较大，需要收费版本才能存下。

为了以后和精调权重再次合并，这个合并模型请从Google Drive中下载一份到本地，以后就可以在本地合并最终模型了，具体原因在文末。

In [ ]:
!cp -fr /content/alpaca-combined /content/drive/MyDrive/

# 直接从driver load

## `注意，除第一次外都执行以下步骤`

拷贝模型文件

In [ ]:
!cp -r /content/drive/MyDrive/alpaca-combined /content/

拷贝精调文件

注意：精调文件这里假设是ft.json，可以根据自己的文件名，调整第一个ft.json，目标名称请不要改动，以便后续步骤中对齐

In [ ]:
!mkdir -p datas
!cp /content/drive/MyDrive/ft.json /content/datas/ft.json

## 进行进一步精调


In [ ]:
!pip install datasets
!pip install deepspeed

执行前，请完成以下设置：

* 在菜单：代码执行程序中点击更改运行时类型
* 在硬件加速器中选择GPU
* 在GPU类型中选择合适的类型，我一般使用A100，其他型号似乎内存不足。

在我的使用过程中，一次精调大概20分钟左右，花费点数5-6个点，100个点9.99，一次大概合人民币3，4块钱吧，当然这个取决于你的数据量大小

In [ ]:
!cd ./Chinese-LLaMA-Alpaca/scripts && python run_clm_sft_with_peft.py --deepspeed ds_zero2_no_offload.json --model_name_or_path /content/alpaca-combined --tokenizer_name_or_path /content/alpaca-combined --dataset_dir /content/datas --validation_split_percentage 0.001 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --do_train --do_eval --seed $RANDOM --max_steps 100 --lr_scheduler_type cosine --learning_rate 1e-4 --warmup_ratio 0.03 --weight_decay 0 --logging_strategy steps --logging_steps 10 --save_strategy steps --save_total_limit 3 --evaluation_strategy steps --eval_steps 250 --save_steps 500 --gradient_accumulation_steps 1 --preprocessing_num_workers 8 --max_seq_length 512 --output_dir /content/chinese_sfted --overwrite_output_dir --ddp_timeout 30000 --logging_first_step True --lora_rank 8 --lora_alpha 32 --trainable "q_proj,v_proj,k_proj,o_proj,gate_proj,down_proj,up_proj" --modules_to_save "embed_tokens,lm_head" --lora_dropout 0.05 --fp16 --torch_dtype float16 --validation_file /content/datas/ft.json --force_resize_embeddings False --gradient_checkpointing --ddp_find_unused_parameters False

# 收尾

生成模型放到了chinese_sfted文件夹里面

将生成的模型再次拷贝回Google Drive，后续就可以进行下载了。这个生成模型是Lora权重，实际使用的时候有两种办法：

* 模型加载时，加载原合并模型，但另外指定--lora参数加载lora权重，这种办法较慢，但可以用于过程中对精调模型进行验证
* 和原合并模型再次合并，合并后作为完整模型使用。

不管是上诉哪种使用办法，建议都是将lora模型拷贝回本地进行合并，一是因为合并的设备要求不高，在本地就可以完成，另外是因为lora模型的体量较完整模型小，我这里7B版本的原始合并模型在16G左右，精调权重大概1.5G，差很多，这样，在国内环境下，下载精调模型更容易一些。

*预先对输出文件名进行更改，原因见下一段，这里预先执行主要是因为如果移动后改名的话，google drive会把mv前后的文件都自动下载，造成资源的浪费*


***deperated：***`最新版Chinese-LLaMA-Alpaca已经自动完成了格式转换，该步骤已不需要执行`


---

In [ ]:
!mv /content/chinese_sfted/pytorch_model.bin /content/chinese_sfted/adapter_model.bin

`以下拷贝目录和拷贝zip包操作2选1`

选项1：拷贝结果目录

In [ ]:
!cp -fr /content/chinese_sfted/sft_lora_model /content/drive/MyDrive/

选项2：
可以直接打包后再拷贝到google drive,方便之后的下载等操作

In [ ]:
!zip -j -r /content/chinese_sfted.zip /content/chinese_sfted/sft_lora_model

In [ ]:
!cp -fr /content/chinese_sfted.zip /content/drive/MyDrive/

# 关于产出物

***deperated：***`最新版Chinese-LLaMA-Alpaca已经自动完成了格式转换，该步骤已不需要执行`


---


产出物为Lora权重，目录结构中包含pytorch_model.bin文件，但如果用于合并，实际需要两个额外文件

* adapter_model.bin (将pytorch_model.bin重新命名)
* adapter_config.json (这个配置文件的主体内容如下，其中参数和我们执行上述精调命令时的参数有对应关系执行可以生成配置文件）

如果你调整了参数，请对应调整这里的参数

In [ ]:
f = open("/content/drive/MyDrive/chinese_sfted/adapter_confjg.json", "a")
f.write("""{
  "base_model_name_or_path": "/content/alpaca-combined"
  "bias": "none",
  "enable_lora": null,
  "fan_in_fan_out": false,
  "inference_mode": true,
  "init_lora_weights": true,
  "lora_alpha": 32,
  "lora_dropout": 0.05,
  "merge_weights": false,
  "modules_to_save": [
    "embed_tokens",
    "lm_head"
  ],
  "peft_type": "LORA",
  "r": 8,
  "target_modules": [
    "q_proj",
    "v_proj",
    "k_proj",
    "o_proj",
    "gate_proj",
    "down_proj",
    "up_proj"
  ],
  "task_type":"CAUSAL_LM"
}""")
f.close()